# Assignment 1

## Task 1 - use Jaccard similarity

In [1]:
import gzip
from collections import defaultdict

In [2]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [74]:
u_bought = defaultdict(set)
i_be_bought = defaultdict(set)
u_bought_rating = defaultdict(list)
i_bought_rating = defaultdict(list)
i_category = defaultdict(list)
for l in readGz("train.json.gz") :
    u_bought[l['reviewerID']].add((l['itemID']))
    u_bought_rating[l['reviewerID']].append([l['itemID'],l['rating']])
    i_be_bought[l['itemID']].add((l['reviewerID']))
    i_bought_rating[l['itemID']].append([l['reviewerID'],l['rating']])
    if l['categories'] not in i_category[l['itemID']] :
        i_category[l['itemID']].append(l['categories'])

In [58]:
def jaccard(u,v) :
    set1 = u_bought[u]
    set2 = u_bought[v]
    return len(set1 & set2)/len(set1 | set2)

In [59]:
##########
for i in i_be_bought :
    print(i_be_bought[i])
    break

{'U347526870', 'U624255242', 'U401058838', 'U842183958', 'U470661329', 'U832439170', 'U573106454', 'U906260102', 'U809095613', 'U478192399', 'U770410694', 'U762998081', 'U263847733', 'U429360487', 'U746303715', 'U408587873', 'U152787439', 'U236951601', 'U490934656'}


In [60]:
def jaccard_i(i1,i2) :
    set1 = i_be_bought[i1]
    set2 = i_be_bought[i2]
    for t in i_category[i1] :
        if t in i_category[i2] :
            return 1
    return len(set1 & set2)/len(set1 | set2)

In [64]:
def jaccard_i_u(i,u) :
    u_buy = u_bought_rating[u]
    sim = 0
    sim2 = 0
    for item in u_buy :
        if jaccard_i(item[0],i) :
            return 1
    return 0

In [65]:
######
for i in u_bought :
    print(u_bought[i])
    break

{'I296399509', 'I330290793', 'I402344648'}


In [66]:
predictions = open("predictions_purchase_ass1_1.txt", 'w')
count =0
for l in open("pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  s = jaccard_i_u(i,u)
  if s>0 :
    predictions.write(u + '-' + i + ',' + str(1) + '\n')
    count+=1
  else :
    predictions.write(u + '-' + i + ',' + str(0) + '\n')

predictions.close()

In [67]:
count

4458

### use validation set

In [70]:
count=0

u_bought = defaultdict(set)
i_be_bought = defaultdict(set)
u_bought_rating = defaultdict(list)
i_bought_rating = defaultdict(list)
i_category = defaultdict(list)
### already purchased data for validation ( should predicted positive )
validation_set1 = defaultdict(list)
user_items = defaultdict(list)
for l in readGz("train.json.gz"):
  count+=1   
  if count<=100000 :
    u_bought[l['reviewerID']].add((l['itemID']))
    u_bought_rating[l['reviewerID']].append([l['itemID'],l['rating']])
    i_be_bought[l['itemID']].add((l['reviewerID']))
    i_bought_rating[l['itemID']].append([l['reviewerID'],l['rating']])
    if l['categories'] not in i_category[l['itemID']] :
        i_category[l['itemID']].append(l['categories'])
  else :
     validation_set1[l['reviewerID']].append(l['itemID'])

In [71]:
import random
### generate randomly 100000 unpurchased user/item pairs

def generate_random_unpurchased_pair(set1,users,items) :
    set2 = defaultdict(list)
    count = 0
    while count < 100000 :
      uID = users[random.randint(0,len(users)-1)]
      iID = items[random.randint(0,len(items)-1)]
      if (not uID in set1 or not iID in 
          set1[uID]) and (not uID in set2 or not iID in set2[uID]) :
        set2[uID].append(iID)
        count += 1
    return set2

### another half validation set that should predicted negative
validation_set2 = generate_random_unpurchased_pair(user_items, user_set, item_set)

In [73]:
### the performance (accuracy) of the baseline model on the validation set
err = 0
for i in validation_set1 :
    for j in validation_set1[i] : 
        if jaccard_i_u(j,i)<=0 :
            err += 1
for i in validation_set2 :
    for j in validation_set2[i] :
        if jaccard_i_u(j,i)>0 :
            err += 1
            
accuracy = (200000 - err)/200000
print("the accuracy of baseline model is "+str(accuracy))



###### 比hw3-3略高，但是小于baseline要低，在kaggle上要高一点


the accuracy of baseline model is 0.54988


## Try second - use user similarity

In [76]:
def jaccard_u(i,u) :
    u_buy_i = i_be_bought[i]
    sim = 0
    for t in u_buy_i :
        sim += jaccard(t,u)
    if sim == 0:
        return 0
    else : return 1

In [77]:
predictions = open("predictions_purchase_ass1_2.txt", 'w')
count =0
for l in open("pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  s = jaccard_u(i,u)
  if s>0 :
    predictions.write(u + '-' + i + ',' + str(1) + '\n')
    count+=1
  else :
    predictions.write(u + '-' + i + ',' + str(0) + '\n')

predictions.close()